# Particle simulation vs Langevin model

In [1]:
include("../src/bmparticles.jl")
include("../src/bmtheory.jl")
using .BParts
using .Theorist
using DifferentialEquations, Distributions

┌ Info: Recompiling stale cache file /Users/nathanielmonpere/.julia/compiled/v1.2/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1240


In [2]:
using Plots
# plotly()
gr()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/nathanielmonpere/.julia/packages/Plots/rNwM4/src/backends.jl:363


Plots.PlotlyBackend()

## Fixed Density

First we compare the particle simulation with the simulations of the Langevin equation (and some analytical results from it) for a fixed population size.

### Parameters for simulation

In [3]:
function extendParams!(arenaParams::Dict)
    bounds = arenaParams["bounds"]
    arenaParams["volume"] = abs(bounds[1][2]-bounds[1][1])*abs(bounds[2][2]-bounds[2][1])
    arenaParams["bperiod"] = [abs(bounds[1][2]-bounds[1][1]), abs(bounds[2][2]-bounds[2][1])]
end

extendParams! (generic function with 1 method)

In [4]:
arenaParams = 
    Dict(
        "n0"=>600,
        "evolveTime"=>2000,
        "bounds"=>((0.,20.),(0.,20.)), 
        "radius"=>0.08, 
        "speed"=>0.05
    )
extendParams!(arenaParams)
evolveTime = arenaParams["evolveTime"]
arenaParams

Dict{String,Any} with 7 entries:
  "volume"     => 400.0
  "evolveTime" => 2000
  "speed"      => 0.05
  "radius"     => 0.08
  "bounds"     => ((0.0, 20.0), (0.0, 20.0))
  "bperiod"    => [20.0, 20.0]
  "n0"         => 600

In [5]:
thermVals = Theorist.thermalValues(arenaParams)

Dict{Any,Any} with 6 entries:
  "γ"  => 0.0339411
  "σc" => 0.32
  "D"  => 5.4019e-5
  "l"  => 1.47314
  "E"  => 0.00159155
  "n"  => 1.5

### Simulations

run a particle simulation:

In [6]:
arena, pos_t_dim_id, vel_t_dim_id, cells_T_ID = 
    BParts.randArenaEvolve(arenaParams["n0"], evolveTime, arenaParams);

println("energy per cell: ",
        BParts.kineticEnergy(arena) / arenaParams["n0"]
    )

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


energy per cell: 0.0015875301878249253


Run an ensemble of Langevin equations:

In [7]:
langevinEnsemble = Theorist.runLangevinSims(1000, arenaParams);

### Analysis

Speed distributions:

For a well mixed population the particle speeds are predicted to be Rayleigh distributed with shape parameter $\sigma = \sqrt{E}$ where $E$ is the average particle energie.

In [8]:
speedPar_t_id = BParts.speedCalc(vel_t_dim_id)
speedLan_t_id = Theorist.speedCalc(langevinEnsemble, 1:evolveTime)
rDistPred = Rayleigh( sqrt(thermVals["E"]) );

In [15]:
p1 =
histogram(vec(speedPar_t_id), bins=100, normalize=true, label="particle simulation")
histogram!(vec(speedLan_t_id), bins=100, normalize=true, label="Langevin simulation")
plot!(range(0,0.25, length=100), pdf.(rDistPred, range(0,0.25, length=100)), linestyle=:dash, linewidth=2, label="predicted")
xlabel!("speed")
ylabel!("distribution")
display(p1)

<!DOCTYPE html>
 
 
 Plots.jl

While the particle simulation agrees nicely with the predicted Rayleigh distribution, the Langevin simulations appear to deviate slightly. I haven't quite figured out what is causing this...

Velocity Autocorrelation:

In [16]:
corrTime = 200
timesCorr_t, vCorrLan_t = Theorist.velCorrelation(langevinEnsemble, (1,corrTime), steps=corrTime);
vCorrPar_t = BParts.velocityAutocorrelation(vel_t_dim_id[1:corrTime, :, :]);
vCorrPred_t = map( t->2*thermVals["E"]*exp(-thermVals["γ"]*t), 0:corrTime-1 );

In [14]:
p1 = plot(timesCorr_t, vCorrLan_t, label="Langevin simulation")
plot!(timesCorr_t, vCorrPar_t, label="particle simulation")
plot!(timesCorr_t, vCorrPred_t, label="analytical prediction")
display(p1)

<!DOCTYPE html>
 
 
 Plots.jl

Mean squared displacement:

In [12]:
msdTime = 400
timesMSD_t, msdLan_t = Theorist.msd(langevinEnsemble, arenaParams,  (0, msdTime-1), msdTime);
msdPar_t = BParts.meanSquaredDisplacement(pos_t_dim_id[1:msdTime,:,:], arenaParams["bperiod"]);

In [13]:
p2 = plot(timesMSD_t, msdLan_t, label="Langevin simulation")
plot!(timesMSD_t, msdPar_t, label="particle simulation")
display(p2)

<!DOCTYPE html>
 
 
 Plots.jl

The msd becomes sublinear after a particular distance due to the periodic boundaries.